#Создание модели болталки

## Формирование обучающего датасета

In [1]:
from google.colab import output

!pip install datasets

from datasets import load_dataset

# Загрузка набора данных
dataset = load_dataset("SiberiaSoft/SiberianPersonaChat")

# Получение доступа к конкретным частям набора данных (например, тренировочным данным)
train_dataset = dataset["train"]

# SiberiaSoft/SiberianPersonaChat,
# author = Denis Petrov, Ivan Ramovich
# url    = https://huggingface.co/datasets/SiberiaSoft/SiberianPersonaChat
# year   = 2023

output.clear()

input = train_dataset['input']
output = train_dataset['output']

In [ ]:
# Превращаем тексты в числовые векторы
from sklearn.feature_extraction.text import TfidfVectorizer
# Создаем объект которые будет создавать числовые векторы
vectorizer = TfidfVectorizer()
# "Обучаем" его на всех контекстах, запоминаем частоту каждого слова
vectorizer.fit(input)
# записываем в матрицу, сколько раз каждое слово встречалось в каждом тексте
matrix = vectorizer.transform(input)
print(f'Было:\t{matrix.shape}')

# Сокращаем размерность матрицы
# Импортируем метод главных компонент
from sklearn.decomposition import TruncatedSVD
n_features = 300
svd = TruncatedSVD(n_components=n_features if n_features != 0 else matrix.shape[1])
svd.fit(matrix)
matrix = svd.transform(matrix)
# в результате наблюдений столько же, но столбцов меньше
print(f'Стало:\t{matrix.shape}')
# выводим процент сохраненной информации
print(f'Сохранилось {round(svd.explained_variance_ratio_.sum()*100, 2)} % информации')

import pickle
# Сохранение объекта в файл
with open('/content/drive/MyDrive/GeekBrains/Обработка текста/vectorizer_gpt_dialogs.pkl', 'wb') as vectorizer_file:
    pickle.dump(vectorizer, vectorizer_file)
with open('/content/drive/MyDrive/GeekBrains/Обработка текста/matrix_gpt_dialogs.pkl', 'wb') as matrix_file:
    pickle.dump(matrix, matrix_file)
with open('/content/drive/MyDrive/GeekBrains/Обработка текста/svd_gpt_dialogs.pkl', 'wb') as svd_file:
    pickle.dump(svd, svd_file)

## Обучение модели

In [2]:
from tqdm import tqdm
from time import time
import joblib
import numpy as np
from sklearn.neighbors import BallTree
from sklearn.base import BaseEstimator
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import pickle
main_path = '/content/drive/MyDrive/GeekBrains/Обработка текста'
# Загрузка объекта из файла
with open(main_path+'/vectorizer_gpt_dialogs.pkl', 'rb') as vectorizer_file:
    vectorizer = pickle.load(vectorizer_file)
with open(main_path+'/matrix_gpt_dialogs.pkl', 'rb') as matrix_file:
    matrix = pickle.load(matrix_file)
with open(main_path+'/svd_gpt_dialogs.pkl', 'rb') as svd_file:
    svd = pickle.load(svd_file)

class NeigborSampler(BaseEstimator):
  def __init__(self, k=5, temperature=1.0):
    self.k = k
    self.temperature = temperature
  def fit(self, X, y):
    self.tree_ = BallTree(X)
    self.y_ = np.array(y)
  def predict(self, X, rendom_state=None):
    softmax = lambda x:np.exp(-x) / sum(np.exp(-x))
    distances, indices = self.tree_.query(X, return_distance=True, k=self.k)
    return self.y_[indices[0][np.array(distances).argmax()]]


from sklearn.pipeline import make_pipeline
model = NeigborSampler(k=5, temperature=0.5)
n_records = 100000
start = time()
model.fit(matrix[:n_records], output[:n_records])
print(f'Обучение модели заняло {time()-start:.2f} секунд')
pipe = make_pipeline(vectorizer, svd, model)
# Сохранение объекта в файл
model_path = '/content/model_NeigborSampler_V2.pkl'
joblib.dump(pipe, model_path)


import random
def check_model(model, input, output):
    index = [i for i in range(len(input))]
    random.shuffle(index)
    index = index[:100]
    x_list = [input[i] for i in index]
    y_list = [output[i] for i in index]

    y_pred = [model.predict([x]) for x in tqdm(x_list)]

    tfidf_vectorizer = TfidfVectorizer()
    accuracy = []
    for index in range(len(x_list)):
      tfidf_matrix = tfidf_vectorizer.fit_transform([y_list[index], y_pred[index]])
      accuracy.append(cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0])
    return sum(accuracy)/len(accuracy)*100
print('Начата оценка модели')
print('Точность модели на 100 рандомных запросах составила: ', check_model(pipe, input, output))

# k=1,  temperature=0.5/1.0/1.5 --- Отвечает хорошо, но на сложные вопросы отвечает не по теме. Ответы одинаковые.

# k=5,  temperature=0.5 --- На некоторые сложные вопросы отвечает не по теме, ответы менее эмоционально окрашены, задает вопросы в ответ
# k=5,  temperature=1.0 --- Ближе к теме даже по сложным вопросам
# k=5,  temperature=1.5 --- На некоторые сложные вопросы отвечает не по теме, ответы более эмоционально окрашены

# k=10, temperature=0.5 --- Сильнее галюцинирует
# k=10, temperature=1.0 --- Начинает галюцинировать
# k=10, temperature=1.5 --- Единичные галюцинирования, более эмоционально окрашенные ответы


# точность на 100 рандомных примерах

# n_features = 300
# NeigborSampler(k=5, temperature=0.5)
# model_NeigborSampler_b0 (0 - 10000)     - 3.612674066215919
# model_NeigborSampler_b1 (10000 - 20000) - 3.157299485290247
# model_NeigborSampler_b2 (20000 - 30000) - 2.7059970034305003

Обучение модели заняло 10.07 секунд


100%|██████████| 100/100 [00:38<00:00,  2.60it/s]


Точность модели на 100 рандомных запросах составила:  3.1341488367505934


#Написание чат-бота
@grey_fiin_TestTGBot

In [35]:
from google.colab import output
import textwrap
from tqdm import tqdm
import time
import os

!pip install telebot
from datetime import datetime  # модуль для работы с датой
import telebot  # модуль для работы с ботом телеграмм
from telebot import types

!pip install pymorphy2
import pymorphy2
# Создание экземпляра объекта класса MorphAnalyzer
morph = pymorphy2.MorphAnalyzer()
import re
import requests


output.clear()

##1-ый сценарий болталка

In [40]:
def get_answer(p):
  return pipe.predict([p])

##2-ой погода в городе

In [36]:
def check_1(message):
  if re.search(r'погод', message.lower()):
    for city in message.split():
      city = morph.parse(city)[0].normal_form
      url = 'https://api.openweathermap.org/data/2.5/weather?q='+city+'&units=metric&lang=ru&appid=79d1ca96933b0328e1c7e3e7a26cb347'
      weather = requests.get(url).json()
      if weather['cod']!='404':
        weather = f"На улице сейчас: {weather['weather'][0]['description']}\nТемпература {weather['main']['temp']} °C, но ощущается как {weather['main']['feels_like']} °C"
        return weather
  else:
    return None

##3-ий гугл поиск

In [ ]:
from googlesearch import search
import requests
from bs4 import BeautifulSoup
import re

def check_2(message):
  if re.search(r'поиск|найди|найти', message.lower()):
    query = re.sub(r'поиск|найди|найти', '', message.lower())
    results = ''
    for url in search(query, stop=10):
      try:
        res = requests.get(url)
        soup = BeautifulSoup(res.text, 'html.parser')
        desc = soup.find('meta', {'name': 'description'})['content']
        if re.search(r'[А-Яа-яЁё]', desc):
          results+=url+'\n'+desc+'\n\n'
      except:
         pass
    if len(results)==0:
      return 'Возникла ошибка поиска'
    else:
      return results
  else:
    return None

##TeleBot

In [74]:
token = '2063078354:AAHirg8QiAEX26sMyB-pCIT3MkSuPPfd4UM'  # уникальный токен бота!!!
bot = telebot.TeleBot(token)  # создаем имя бота и отправляем ему токен

# Функция, обрабатывающая команду /start
@bot.message_handler(commands=["start"])
def start(message, res=False):
    hello_message = '''
    Привет! Это бот-болталка.\n
    Я умею отвечать на твои сообщения (правда не всегда подходяще...)\n
    Также я могу отправить погоду в текущий момент, просто скажи "погода" и город в котором надо искать\n
    А еще я могу искать в гугле)
    '''
    bot.send_message(message.chat.id, hello_message)

# Получение сообщений от юзера и возврат ответа
@bot.message_handler(content_types=["text"])
def handle_text(message):
    c1 = check_1(message.text)
    c2 = check_2(message.text)
    if c1:
      bot.send_message(message.chat.id, c1)
    elif c2:
      bot.send_message(message.chat.id, c2)
    else:
      t = message.text # получаем сообщение пользователя
      answer = get_answer(t) # получаем ответ модели
      bot.send_message(message.chat.id, answer)

# Запускаем бота
bot.polling(none_stop=False, interval=0)